# Initial Data Exploration -- 1
Wir haben uns entschieden, welche Daten wir für unser Projekt verwenden wollen.
Um nicht mit Corona beeinflussten Daten zu arbeiten, wollen wir die NYC Taxi Data aus 2019 nehmen.
Daher beginnt nun die initiale Datenanalyse-Phase, in der wir uns die Daten anschauen, Outlier finden wollen und die Daten so "Clean" machen wollen.
Die Datenanalyse ist dabei in zwei Notebooks gesplittet.
Für die Analyse benötigen wir folgenden Imports:

In [1]:
import dask.dataframe as dd
# Verwendung von Dask für eine schnellere Datenanalyse und -vorverarbeitung
# Pandas ähnlich aber mit parallel Computing und damit für große Datenmenge geeignet
# Vergleich: https://medium.com/featurepreneur/pandas-vs-dask-the-power-of-parallel-computing-994a202a74bd

import pandas as pd
# für kleinere Datenrahmen, bei denen es nicht so sehr auf die Geschwindigkeit ankommt, noch Pandas zu verwenden
# und für das Plotten von Diagrammen

import pyarrow.parquet as pq
# pyarrow zum Lesen großer parquet Files
import datetime

import time

import numpy as np


import os
# os zum Arbeiten mit verschiedenen Dateien in einem Verzeichnis und zum Ausgeben von Informationen über Dateien

import warnings
# diese Lib hilft, einige Warnungen zu ignorieren, wenn man z.B. Dataframes lädt
warnings.filterwarnings("ignore")

# Datenübersicht
Wir haben alle yellow taxi drives von Januar 2019 bis Dezember 2019 gesammelt.
Zum Ausführen des gesamten Notebooks werden daher die parquet files von januar bis dezember im Ordner: data/raw/parquet benötigt
Für die nächste Zelle werden alle parquet Files benötigt, für die restliche Datenanalyse reicht die Datei aus Januar 2019:
yellow_tripdata_2019-01.parquet

Heruntergeladen werden 12 Dateien in parquet Format, über die wir folgende Übersicht haben:

In [2]:
df_file_summary = pd.DataFrame()
for filename in os.listdir('../data/raw/parquet/'):
    if filename.endswith(".parquet"):
        df_month_data = pq.read_table('../data/raw/parquet/' + os.path.join(filename), columns=[])
        size = str(round(os.stat('../data/raw/parquet/' + os.path.join(filename)).st_size / 1024 / 1024, 2)) + ' mb'
        new_row = {
            'file_name': filename,
            'file_size': size,
            'number_of_records': df_month_data.num_rows,
        }
        df_file_summary = df_file_summary.append(new_row, ignore_index=True)
df_file_summary

,file_name,file_size,number_of_records
0,yellow_tripdata_2019-01.parquet,105.32 mb,7696617
1,yellow_tripdata_2019-02.parquet,98.57 mb,7049370
2,yellow_tripdata_2019-03.parquet,110.64 mb,7866620
3,yellow_tripdata_2019-04.parquet,105.04 mb,7475949
4,yellow_tripdata_2019-05.parquet,106.31 mb,7598445
5,yellow_tripdata_2019-06.parquet,98.14 mb,6971560
6,yellow_tripdata_2019-07.parquet,89.53 mb,6310419
7,yellow_tripdata_2019-08.parquet,85.83 mb,6073357
8,yellow_tripdata_2019-09.parquet,92.61 mb,6567788
9,yellow_tripdata_2019-10.parquet,101.37 mb,7213891


# Data Cleaning
Für die Datenaufbereitung müssen wir uns die verschiedenen Merkmale ansehen.
Weil die Datenmenge so groß ist machen wir keine Analyse für jeden Monat, sondern nehmen die Daten aus dem Januar 2019 repräsentativ für alle Monate:
Wir betrachten diesen Monat, um einen Einblick in die Daten zu bekommen.
Zunächst konvertieren wir die parquet File in eine csv-Datei.
Später wird das vom Skript csv_converter für alle parquet files übernommen

In [3]:
# df = pd.read_parquet('../data/raw/parquet/' + os.path.join('yellow_tripdata_2019-01.parquet'))
# df.to_csv('../data/raw/csv/' + 'yellow_tripdata_2019-01' + '.csv', index=False)

In [4]:
month = dd.read_csv('../data/raw/csv/yellow_tripdata_2019-01.csv')
print(month.columns)

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')


In [5]:
month.head(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1.0,1.5,1.0,N,151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,NaN,NaN
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1.0,2.6,1.0,N,239,246,1,14.0,0.5,0.5,1.00,0.0,0.3,16.30,NaN,NaN
2,2,2018-12-21 13:48:30,2018-12-21 13:52:40,3.0,0.0,1.0,N,236,236,1,4.5,0.5,0.5,0.00,0.0,0.3,5.80,NaN,NaN
3,2,2018-11-28 15:52:25,2018-11-28 15:55:45,5.0,0.0,1.0,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,7.55,NaN,NaN
4,2,2018-11-28 15:56:57,2018-11-28 15:58:33,5.0,0.0,2.0,N,193,193,2,52.0,0.0,0.5,0.00,0.0,0.3,55.55,NaN,NaN


## Datenaufbereitung
Für die Datenanalyse macht es Sinn an der ein oder anderen Stelle Werte zu ergänzen.
Deswegen ergänzen wir einige Werte:
<ul>
  <li>trip_times (min) --> Die Fahrzeit in Minuten</li>
  <li>pickup_times (unix) --> Die PickUp Zeit in Unix konvertiert</li>
  <li>Speed (mph) --> Die Geschwindkeit in Miles per Hour</li>
</ul>

In [6]:
def convert_to_unix(s):
    return time.mktime(datetime.datetime.strptime(s, '%Y-%m-%d %H:%M:%S').timetuple())


def return_with_trip_times(month):
    duration = month[['tpep_pickup_datetime','tpep_dropoff_datetime']].compute()
    duration_pickup = [time.mktime(datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').timetuple()) for x in duration['tpep_pickup_datetime'].values]
    duration_drop = [time.mktime(datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').timetuple()) for x in duration['tpep_dropoff_datetime'].values]
    durations = (np.array(duration_drop) - np.array(duration_pickup))/float(60)
    new_frame = month.compute()

    new_frame['trip_times (min)'] = durations
    new_frame['pickup_times (unix)'] = duration_pickup
    new_frame['Speed (mph)'] = 60*(new_frame['trip_distance']/new_frame['trip_times (min)'])

    return new_frame

frame_with_durations = return_with_trip_times(month)
frame_with_durations.head(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,trip_times (min),pickup_times (unix),Speed (mph)
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1.0,1.5,1.0,N,151,239,1,...,0.5,1.65,0.00,0.3,9.95,NaN,NaN,6.666667,1.546300e+09,13.500000
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1.0,2.6,1.0,N,239,246,1,...,0.5,1.00,0.00,0.3,16.30,NaN,NaN,19.200000,1.546301e+09,8.125000
2,2,2018-12-21 13:48:30,2018-12-21 13:52:40,3.0,0.0,1.0,N,236,236,1,...,0.5,0.00,0.00,0.3,5.80,NaN,NaN,4.166667,1.545397e+09,0.000000
3,2,2018-11-28 15:52:25,2018-11-28 15:55:45,5.0,0.0,1.0,N,193,193,2,...,0.5,0.00,0.00,0.3,7.55,NaN,NaN,3.333333,1.543417e+09,0.000000
4,2,2018-11-28 15:56:57,2018-11-28 15:58:33,5.0,0.0,2.0,N,193,193,2,...,0.5,0.00,0.00,0.3,55.55,NaN,NaN,1.600000,1.543417e+09,0.000000
5,2,2018-11-28 16:25:49,2018-11-28 16:28:26,5.0,0.0,1.0,N,193,193,2,...,0.5,0.00,5.76,0.3,13.31,NaN,NaN,2.616667,1.543419e+09,0.000000
6,2,2018-11-28 16:29:37,2018-11-28 16:33:43,5.0,0.0,2.0,N,193,193,2,...,0.5,0.00,0.00,0.3,55.55,NaN,NaN,4.100000,1.543419e+09,0.000000
7,1,2019-01-01 00:21:28,2019-01-01 00:28:37,1.0,1.3,1.0,N,163,229,1,...,0.5,1.25,0.00,0.3,9.05,NaN,NaN,7.150000,1.546298e+09,10.909091
8,1,2019-01-01 00:32:01,2019-01-01 00:45:39,1.0,3.7,1.0,N,229,7,1,...,0.5,3.70,0.00,0.3,18.50,NaN,NaN,13.633333,1.546299e+09,16.283619
9,1,2019-01-01 00:57:32,2019-01-01 01:09:32,2.0,2.1,1.0,N,141,234,1,...,0.5,1.70,0.00,0.3,13.00,NaN,NaN,12.000000,1.546301e+09,10.500000


# Datenaufbereitung für die erste Analyse abgeschlossen
Damit wurde die CSV aus 2019 ergänzt um die benötigten Werte für die Analyse und wird jetzt abgespeichert im interim Folder
Hier hilft später auch das Skript zum data_cleaning abhilfe

In [ ]:
# frame_with_durations.to_csv('../data/interim/yellow_tripdata_2019-01_durations.csv', index=False)